In [7]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

## Question 1

On trouve sur Wikipédia ( https://en.wikipedia.org/wiki/Proton_therapy#Description ) que la gamme d'énergie est typiquement de 70 à 250 MeV. La plus grande énergie utilisée est donc de 250 MeV.



## Question 2

* Trouver comment exprimer la densité électronique $n_e$ en fonction de la composition atomique (Z?) et de $\rho$
* Calculer les densités électroniques de l'eau et de l'os cortical?
* Tracer les courbes de stopping power à partir de PSTAR

## Question 3

* Analyse dimensionnelle (vérifier les unités)
* S'assurer que les unités sont les mêmes des deux côtés de l'équation

## Question 4

## Question 5

## Question 6

## Question 7

## Question 8

## Question 9

## Question 10

## Question 11

## Question 12

## Question 13

## Question 14